In [1]:
# TODOS:
# Add warnings if expected directory or files do not exist
# Comment heavily

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import ecephys_analyses as ea

ea.get_channels("CNPIX12-Santiago", "novel_objects_deprivation", "imec1", "hippocampus")

array([346, 348, 350, 352, 354, 356, 358, 360, 362, 364, 366, 368, 370,
       372, 374, 376, 378, 380, 382,   1,   3,   5,   7,   9,  11,  13,
        15,  17,  19,  21,  23,  25,  27,  29,  31,  33,  35,  37,  39,
        41,  43,  45,  47,  49,  51,  53,  55,  57,  59,  61,  63,  65,
        67,  69,  71,  73,  75,  77,  79,  81,  83,  85,  87,  89,  91,
        93,  95,  97,  99, 101, 103, 105, 107, 109, 111, 113, 115, 117,
       119, 121, 123, 125, 127, 129, 131, 133, 135, 137, 139, 141, 143,
       145, 147, 149, 151, 153, 155, 157, 159, 161, 163, 165, 167, 169,
       171, 173, 175, 177, 179, 181, 183, 185, 187, 189])

In [4]:
paths = ea.get_lfp_bin_paths("CNPIX12-Santiago", "novel_objects_deprivation", alias="recovery_sleep", probe="imec1")
print(paths)

[PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX12-Santiago/11-1-2021/SpikeGLX/11-1-2021_g1/11-1-2021_g1_imec1/11-1-2021_g1_t0.imec1.lf.bin')
 PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX12-Santiago/11-1-2021/SpikeGLX/11-1-2021_g1/11-1-2021_g1_imec1/11-1-2021_g1_t1.imec1.lf.bin')
 PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX12-Santiago/11-1-2021/SpikeGLX/11-1-2021_g1/11-1-2021_g1_imec1/11-1-2021_g1_t2.imec1.lf.bin')]


In [11]:
ea.get_project_counterparts("SPWRs", "CNPIX12-Santiago", paths, "spg.nc")

[PosixPath('/Volumes/neuropixel_analysis/Data/SPWRs/CNPIX12-Santiago/11-1-2021/SpikeGLX/11-1-2021_g1/11-1-2021_g1_imec1/11-1-2021_g1_t0.imec1.spg.nc'),
 PosixPath('/Volumes/neuropixel_analysis/Data/SPWRs/CNPIX12-Santiago/11-1-2021/SpikeGLX/11-1-2021_g1/11-1-2021_g1_imec1/11-1-2021_g1_t1.imec1.spg.nc'),
 PosixPath('/Volumes/neuropixel_analysis/Data/SPWRs/CNPIX12-Santiago/11-1-2021/SpikeGLX/11-1-2021_g1/11-1-2021_g1_imec1/11-1-2021_g1_t2.imec1.spg.nc')]

# Under development

In [313]:
# TODO
# Now write functions to get analysis paths, replacing existing path parts and inserting at various points along the hierarchy. 
# Analysis directory should not be specified in raw data yamlspec file. 
# Perhaps analysis directory hierarchy should not mirror SGLX hierarchy. Perhaps most files should just be saved in session folders. Have a session column in the analysis drive manifest. 
# One group of analysis file management functions should get a single subject, experiment, or alias file with an arbitrary name. Another should get a group of SGLX-style named analysis files from a session folder. 
# Later, we can improve on what "SGLX style nameing" means. 
# Add time+datetime support to aliases
# Change files to paths where appropriate
# Or maybe, sessions should just be explicit lists of absolute file paths, and aliases as well. Easy to generate this kind of a file, given the code you've already written. 
# There should be a config.py file for setting analysis/raw data mount points. 
# Functions for analysis file management and for raw data management should be completely separate, even if it results in redundancy. 


# DONE
# Could remove run ordering from yamlspec file entirely. Then ordering is lost for analysis files, and you must have a manifest with file order in the analysis drive. No big deal. 
# Analysis folders (e.g. "spw-analysis") should be above subject folders in the hierarchy. 
# Just get a drop-in replacement working for now. Cut your losses. 


In [5]:
import ecephys_analyses as ea
foo = ea.get_subject_analysis_dir('Adrian')

In [309]:
df = sou.get_files(docs, 'Allan', 'novel_objects_deprivation', alias='light-period-circadian-match', probe='imec0', stream='lf', ftype='bin');

In [310]:
def validate_sglx_path(path):
    (run, gate, trigger, probe, stream, ftype) = fmgmt.parse_sglx_fname(path.name) 
    probe_dir = path.parent
    gate_dir = probe_dir.parent
    assert probe_dir.name == f"{run}_{gate}_{probe}"
    assert gate_dir.name == f"{run}_{gate}"
    return gate_dir, probe_dir.name, path.name
    
def _get_path_parts(path):
    gate_dir, probe_dirname, fname = validate_sglx_path(path)
    session_dir = gate_dir.parent
    subject_dir = session_dir.parent    
    return subject_dir, session_dir.name, gate_dir.name, probe_dir.name, fname

def get_analysis_counterpart(path, extension, analysis_subject_dir):
    subject_dir, session_dirname, gate_dirname, probe_dirname, fname = _get_path_parts(path)
    (run, gate, trigger, probe, stream, ftype) = fmgmt.parse_sglx_fname(fname)
    new_fname = f"{run}_{gate}.{probe}.{extension}"
    return analysis_subject_dir / session_dirname / gate_dirname / probe_dirname / new_fname

def get_analysis_counterparts(paths, extension, analysis_subject_dir):
    counterparts = [get_analysis_counterpart(p, extension, analysis_subject_dir) for p in paths]
    return list(dict.fromkeys(counterparts))

def get_path_parts(df):
    return pd.DataFrame.from_records(map(_get_path_parts, df.path.values), columns=['subject_dir', 'session_dirname', 'gate_dirname', 'probe_dirname', 'fname'])
    
def add_path_parts(df):
    return pd.concat([df, get_path_parts(df)], axis=1)

In [311]:
analysis_subject_dir = Path('/Volumes/neuropixel_analysis/Data/chronic/CNPIX8-Allan/')

In [312]:
get_analysis_counterparts(df.path.values, 'spg.nc', analysis_subject_dir)

[PosixPath('/Volumes/neuropixel_analysis/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_g0/3-1-2021_A_g1_imec0/3-1-2021_g0.imec0.spg.nc'),
 PosixPath('/Volumes/neuropixel_analysis/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021-a_g0/3-1-2021_A_g1_imec0/3-1-2021-a_g0.imec0.spg.nc'),
 PosixPath('/Volumes/neuropixel_analysis/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g0/3-1-2021_A_g1_imec0/3-1-2021_A_g0.imec0.spg.nc'),
 PosixPath('/Volumes/neuropixel_analysis/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1.imec0.spg.nc'),
 PosixPath('/Volumes/neuropixel_analysis/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_B_g0/3-1-2021_A_g1_imec0/3-1-2021_B_g0.imec0.spg.nc'),
 PosixPath('/Volumes/neuropixel_analysis/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_C_g0/3-1-2021_A_g1_imec0/3-1-2021_C_g0.imec0.spg.nc')]